In [1]:
from notebooks_tools import *
from gbdt_tools import *
%matplotlib inline

# Define parameters
set the GBDTs hyper-parameters

In [2]:
data_type_arr=['MC_BNB','openCOSMIC_MC'] 
nevents_train_arr=[300000,200000] 
parameters = dict({ 
        'evnts_frac':0.01,# events fraction to process
        'debug':2, # prints out information during the processes
        'scale_pos_weight':2., # Balancing of positive and negative weights.
        'objective':'binary:logistic', # Specify the learning task and the corresponding learning objective or a custom objective function to be used      
        'eta':0.025, # Boosting learning rate        
        'eval_metric':'error', # a custom evaluation metric      
        'silent':True, # Whether to print messages while running boosting
        'nthread':6, # Number of parallel threads used to run xgboost.     
        'min_child_weight':4, # Minimum sum of instance weight(hessian) needed in a child.        
        'max_depth':13,# Maximum tree depth for base learners
        'gamma':0.7, # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'colsample_bytree':0.5, # Subsample ratio of columns when constructing each tree
        'subsample':0.8, # Subsample ratio of the training instance       
        'num_class':5, #  If early stopping occurs...     
        'Ntrees':500,        
        'Nfolds':10,
        'num_round':200
#         'reg_alpha':1e-5 # L1 regularization term on weights
        })
# [http://xgboost.readthedocs.io/en/latest/python/python_api.html]
train_filename = []
for data,nevnts in zip(data_type_arr,nevents_train_arr):
    train_filename.append(TrainingSampleFileName( data , nevnts , main_path))    
    print_filename( train_filename[-1] , "input: traininig sample file" )

input: traininig sample file
/Users/erezcohen/Desktop/uBoone/AnalysisTreesAna/TrainingSamples/trainsample_300000_tracks_MC_BNB.csv
input: traininig sample file
/Users/erezcohen/Desktop/uBoone/AnalysisTreesAna/TrainingSamples/trainsample_200000_tracks_openCOSMIC_MC.csv


# comments:
katherine gives in parameter_opt:
eta=0.05
max_depth=9
gamma=0
colsample_bytree=0.8


# Load the data
from the training files

In [3]:
data,label,weight = boost_multiscore.load_data( bnb_mc_filename=train_filename[0] , 
                                               corsika_mc_filename=train_filename[1] ,   
                                               debug=parameters['debug'] ,
                                               feature_names=feature_names ,
                                               tracks_frac=parameters['evnts_frac'] )

took only a fraction of 0.01 of tracks from MC-BNB and CORSIKA samples:
1436 protons, 288 pions, 607 muons, 559 em
2000 cosmic tracks


# cross-validation step
from the training files

In [ ]:
do_cross_validation = yesno('cross validate?')
if do_cross_validation:
    test_error,test_falsepos,test_falseneg,scores = boost_multiscore.run_cv( data , label , weight , parameters )

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(2,2,1)
plt.hist(test_error);
set_axes(ax,x_label='test error',y_label='counts',fontsize=15)

ax = fig.add_subplot(2,2,2)
plt.hist([test_falsepos,test_falseneg],label=['false positive','false negative']);
set_axes(ax,x_label='fraction',y_label='counts',fontsize=15)
plt.legend(fontsize=20,loc='best')

ax = fig.add_subplot(2,1,2)
plt.hist([scores[0],scores[1],scores[2],scores[3],scores[4]],
         label=['$p$-score','$\\mu$-score','$\\pi$-score','$em$-score','$cosmic$-score'],
         histtype='step',linewidth=2);
set_axes(ax,x_label='score',fontsize=15)
plt.legend(fontsize=20,loc='best')

# optimization of the hyper-parameters

In [5]:
do_optimize_paramters = yesno('optimize parameters?')
if do_optimize_paramters:
    test_error,test_falsepos,test_falseneg,scores = boost_multiscore.parameter_opt( data , label , weight , parameters )

optimize parameters? [Y/n] yes


TypeError: cv() got an unexpected keyword argument 'stratified'